# ----------------------------------
# Import modules
# ----------------------------------

In [1]:
import os
import requests                      # HTTP client for API calls
import pandas as pd                  # Tabular data handling
from datetime import datetime        # Datetime handling
from typing import Iterable, Optional, Dict, Union
import matplotlib.pyplot as plt
%pip install yfinance
import yfinance as yf
from src.exchange_rates import get_share_prices_2
from src.debug_print import debug_print
from src.plot_shares import plot_candles_with_volatility
from src.plot_shares_ROI import plot_candles_with_volatility_and_target as plt_vol_trg
from src.plot_shares_ROI2 import plot_candles_volatility_volume_roi as ROI
from src.fetch_lse_tickers import get_ftse100

Note: you may need to restart the kernel to use updated packages.


# ----------------------------------
# Import functions
# ----------------------------------

In [2]:

from src.debug_print import debug_print
from src.exchange_rates import get_exchange_rates, get_crypto_prices, get_fx_and_crypto, get_share_prices, convert_prices_to_base_currency, plot_fx_timeseries

# ----------------------------------
# Set up variables
# ----------------------------------

In [3]:
base_currency = "GBP"
target_currencies = ["USD", "GBP", "EUR", "JPY"]
cryptos = ["BTC", "ETH"]
shares = ['AAPL', 'RR.L', 'MSFT', 'NVDA', 'LDO.MI','4816.T']

start_date = datetime(2024, 6, 1)
purchase_date = datetime(2025, 6, 1)
end_date = pd.Timestamp.today().normalize()

try:
    ftse100 = get_ftse100()
    ftse100["Yahoo_Ticker"] = ftse100["Ticker"] + ".L"

    shares_lse = ftse100["Yahoo_Ticker"].to_list()


except Exception as e:
    print(f"ERROR in ftse100 {type(e).__name__}: {e}")

# --------------------------------
# SHARES' PRICES
# --------------------------------

In [4]:
try:
    df_shares2 = get_share_prices_2(
    tickers=shares_lse,
    start=start_date,
    end=end_date,
    base_currency = base_currency,
    vol_window = 20,
)

    print(f"{debug_print()} get_share_prices_2:\n{df_shares2}")
except Exception as e:
    print(f"{debug_print()} [FAILED] running get_share_prices_2 {type(e).__name__}: {e} ")


[DEBUG] File: C:\Users\ingca\AppData\Local\Temp\ipykernel_13632\2516834727.py, Function: <module>, Line: 12
None [FAILED] running get_share_prices_2 TypeError: 'NoneType' object is not subscriptable 


# ----------- PLOT SHARE ---------------

In [5]:
try:
    actions_list   = df_shares2.columns.get_level_values("ACTION").unique().to_list()
    currencies_list = df_shares2.columns.get_level_values("CURRENCY").unique()
    metrics   = df_shares2.columns.get_level_values("METRIC").unique()

    plot_candles_with_volatility(
        df=df_shares2,
        actions=['RR.L_GBP→GBP'],
        start=df_shares2.index.min(),
        end=df_shares2.index.max()
    )
except Exception as e:
    print(f"{debug_print()} [FAILED] PLOT SHARE {type(e).__name__}: {e} ") 

[DEBUG] File: C:\Users\ingca\AppData\Local\Temp\ipykernel_13632\2553617500.py, Function: <module>, Line: 13
None [FAILED] PLOT SHARE NameError: name 'df_shares2' is not defined 


# ----------- PLOT SHARE - plot_candles_with_volatility_and_target ---------------


In [6]:
try:
    actions_list   = df_shares2.columns.get_level_values("ACTION").unique().to_list()
    currencies_list = df_shares2.columns.get_level_values("CURRENCY").unique()
    metrics   = df_shares2.columns.get_level_values("METRIC").unique()

    plt_vol_trg(
        df=df_shares2,
        actions=['RR.L_GBP→GBP'],
        start=df_shares2.index.min(),
        end=df_shares2.index.max(),
        roi_target=1.57
    )
except Exception as e:
    print(f"{debug_print()} [FAILED] plt_vol_trg {type(e).__name__}: {e} ") 

[DEBUG] File: C:\Users\ingca\AppData\Local\Temp\ipykernel_13632\497573991.py, Function: <module>, Line: 14
None [FAILED] plt_vol_trg NameError: name 'df_shares2' is not defined 


# ----------- PLOT SHARE - plot_candles_volatility_volume_roi ---------------


In [7]:
try:
    from src.plot_shares_ROI2 import plot_candles_volatility_volume_roi as ROI
    
    actions_list   = df_shares2.columns.get_level_values("ACTION").unique().to_list()
    currencies_list = df_shares2.columns.get_level_values("CURRENCY").unique()
    metrics   = df_shares2.columns.get_level_values("METRIC").unique()

    ROI(
        df=df_shares2,
        actions=['RR.L_GBP→GBP'],
        start=df_shares2.index.min(),
        end=df_shares2.index.max(),
        purchase_date='2025-01-01',
        roi_target=0.55
    )
except Exception as e:
    print(f"{debug_print()} [FAILED] plot_candles_volatility_volume_roi {type(e).__name__}: {e} ")


[DEBUG] File: C:\Users\ingca\AppData\Local\Temp\ipykernel_13632\638906002.py, Function: <module>, Line: 17
None [FAILED] plot_candles_volatility_volume_roi NameError: name 'df_shares2' is not defined 


In [8]:
try:
    from src.plot_shares_ROI2_1 import plot_candles_volatility_volume_roi as ROI
    
    actions_list   = df_shares2.columns.get_level_values("ACTION").unique().to_list()
    currencies_list = df_shares2.columns.get_level_values("CURRENCY").unique()
    metrics   = df_shares2.columns.get_level_values("METRIC").unique()

    ROI(
        df=df_shares2,
        actions=['RR.L_GBP→GBP'],
        start=df_shares2.index.min(),
        end=df_shares2.index.max(),
        purchase_date='2025-01-01',
        roi_target=0.55
    )
except Exception as e:
    print(f"{debug_print()} [FAILED] plot_candles_volatility_volume_roi {type(e).__name__}: {e} ")


[DEBUG] File: C:\Users\ingca\AppData\Local\Temp\ipykernel_13632\1422629401.py, Function: <module>, Line: 17
None [FAILED] plot_candles_volatility_volume_roi NameError: name 'df_shares2' is not defined 
